In [6]:
import torch, os, re, pandas as pd, json
import torch
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig

import pandas as pd
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
# from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
from torch import optim
import gc
from torch.nn import DataParallel
import nltk
import random
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
with open('/home/ubuntu/Darwin_Gatsby/gatsby_raw.txt', 'r') as file:
    gatsby = file.read().replace('\n', ' ')

with open('/home/ubuntu/Darwin_Gatsby/origin_of_species_raw.txt', 'r') as file:
    origin = file.read().replace('\n', ' ')

In [8]:
gatsby_sen_list = nltk.sent_tokenize(gatsby)
origin_sen_list = nltk.sent_tokenize(origin)
print(len(gatsby_sen_list))
print(len(origin_sen_list))

2439
3981


In [27]:
import re
import string
from collections import Counter
from collections import defaultdict

def clean_book(book_sen_list):
    clean_book_sen_list = []
    for sen in book_sen_list:
        # ref https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces
        sen = re.sub(r'^\s*|\s\s*', ' ', sen).strip().rstrip().lstrip()
        printable = set(string.printable)
        sen = filter(lambda x: x in printable, sen)
        sen = ''.join(filter(lambda x: x in printable, sen))
        clean_book_sen_list.append(sen)
        
    return clean_book_sen_list

def find_passages(book_sen_list, sentence_starts):
    
    passages = []
    sen_index = -1
    while True:
        sen_index += 1
        if sen_index >= len(book_sen_list):
            break
            
        sen = book_sen_list[sen_index]
        
        use_sentence = False
        for sentence_start in bigram_overlap:
            if sen.startswith(sentence_start):
                use_sentence = True
                break
              
        passage = sen
        if use_sentence:
            while len(passage.split()) < 50:
                sen_index += 1
                sen = book_sen_list[sen_index]
                passage += " " + sen
                
            passages.append(' '.join(passage.split()[:50]))
        
    return passages
                



In [28]:
bigram_overlap = {'The fact', 'And what', 'But no', 'As we', 'The most', 'Do they', 'But when', 'But we', 'Some little', 'But the', 'One of', 'He always', 'And now', 'I see', 'It occurred', 'As soon', 'No one', 'The flowers', 'There is', 'In the', 'In fact,', 'There must', 'I cannot', 'Even when', 'The practical', 'I should', 'In this', 'At the', 'Now it', 'The day', 'As he', 'It was', 'Most of', 'The only', 'At first', 'At a', 'I could', 'Nothing at', 'We could', 'The other', 'I remember', 'The old', 'A few', 'But what', 'In one', 'Now I', 'If we', 'So we', 'The masters', 'Let the', 'All the', 'How the', 'By the', 'But how', 'At this', 'See how', 'On a', 'But it', 'I will', 'When we', 'After the', 'In my', 'And when', 'How much', 'I must', 'I would', 'It makes', 'Not only', 'If there', 'And the', 'But I', 'He would', 'A large', 'To my', 'I believe', 'This was', 'This is', 'The eyes', 'It seems', 'A little', 'And I', 'If it', 'With the', 'I know', 'For several', 'As I', 'Now, in', 'After a', 'And we', 'From the', 'The man', 'I was', 'It is', 'I think', 'The truth', 'If I', 'Not that', 'I have', 'A white', 'But there', 'He has', 'Those who', 'Of course', 'Let us', 'They are', 'And as', 'As the', 'In two', 'I am', 'A new', 'And it', 'But in', 'Even if', 'On the', 'And all', 'All these', 'To the', 'But with', 'And if', 'Now, if', 'A man', 'When I', 'Look at', 'The little', 'I feel', 'He must', 'Were all', 'When the', 'In a', 'For some', 'It might', 'The three', 'The whole', 'How do'}

gatsby_clean_sen_list = clean_book(gatsby_sen_list)
origin_clean_sen_list = clean_book(origin_sen_list)

gatsby_passages = find_passages(gatsby_clean_sen_list, bigram_overlap)
origin_passages = find_passages(origin_clean_sen_list, bigram_overlap)


In [30]:
print(len(gatsby_passages))
print(len(origin_passages))

301
763


In [32]:
import random

gatsby_100_real = random.sample(gatsby_passages, 100)
origin_100_real = random.sample(origin_passages, 100)



In [33]:
all_seqs = []
for seq in gatsby_100_real:
    all_seqs.append(('GATSBY_REAL', seq))
for seq in origin_100_real:
    all_seqs.append(('ORIGIN_REAL', seq))


In [34]:
df = pd.DataFrame(all_seqs, columns = ['Source', 'Sequence'])

df.to_csv('finetuened_real.csv', encoding='utf-8', index = False)

In [54]:
import os

pos_path = "/home/ubuntu/IMDB_train/pos/"
neg_path = "/home/ubuntu/IMDB_train/neg/"


pos_token_list = []
pos_text_list = []
neg_token_list = []
neg_text_list = []

os.chdir(pos_path)
  

for file in os.listdir():
    if not file.endswith('.txt'):
        continue
    with open(pos_path+file, 'r') as f:
        pos_text = f.read()
        # put into text list
        pos_text_list.append(pos_text)
        #tokenize and put into token list
        #pos_token_list.append(base_tokenizer.encode(pos_text, return_tensors = 'pt'))

os.chdir(neg_path)

for file in os.listdir():
    if not file.endswith('.txt'):
        continue
    with open(neg_path+file, 'r') as f:
        neg_text = f.read()
        # put into text list
        neg_text_list.append(neg_text)
        #tokenize and put into token list
        #neg_token_list.append(base_tokenizer.encode(neg_text_list, return_tensors = 'pt'))

In [55]:
from io import StringIO
from html.parser import HTMLParser

# ref https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()
# cleaned = test_text_pos.replace('<br /><br />', ' ')
# print(cleaned)

def clean_imdb(review_list):
    for review in range(len(review_list)):
        cleaned_review = strip_tags(review_list[review])
        review_list[review] = cleaned_review

In [60]:
# print(pos_text_list[0])
# print(neg_text_list[0])

clean_imdb(pos_text_list)
clean_imdb(neg_text_list)

# pos_sen_list = []
# neg_sen_list = []

# print(pos_text_list[0])

# for sen in pos_text_list:
#     pos_sen_list.append(nltk.sent_tokenize(sen))
# for sen in pos_text_list:
#     neg_sen_list.append(nltk.sent_tokenize(sen))
# print(len(pos_sen_list))
# print(len(neg_sen_list))



# pos_clean_sen_list = clean_book(pos_sen_list)
# neg_clean_sen_list = clean_book(neg_sen_list)

pos_passages = find_passages(pos_text_list, bigram_overlap)
neg_passages = find_passages(neg_text_list, bigram_overlap)

# print(pos_passages)
# print(neg_passages)

In [64]:
import random

IMDB_100_real_pos = random.sample(pos_passages, 100)
IMBB_100_real_neg = random.sample(neg_passages, 100)
print(IMDB_100_real_pos)

['I am a VERY big Jim Carrey fan. I laughed my ASS off during Liar Liar and Ace Ventura. I also like him in his serious movies, especially Truman Show. This one is a cross between his VERY funny side, and his serious side. He is of course VERY funny', "I haven't seen much German comedy, but if this film is anything to go by, I'm compelled to see more! The simple but effective storyline takes two very different people on a trip from Germany to Italy after Eva, an unemployed mother of two, discovers that her artist husband is", "There is a scene in Dan in Real Life where the family is competing to see which sex can finish the crossword puzzle first. The answer to one of the clues is Murphy's Law: anything that can go wrong, will go wrong. This is exactly the case for Dan Burns", 'This was the first regular filmed Columbo movie episode but yet it aired as the second, after Steven Spielberg\'s "Columbo: Murder by the Book". It\'s also at the same time among one of the better ones!Bernard L

In [66]:
all_seqs_IMDB = []
for seq in IMDB_100_real_pos:
    all_seqs_IMDB.append(('POS_REAL', seq))
for seq in IMBB_100_real_neg:
    all_seqs_IMDB.append(('NEG_REAL', seq))
print(all_seqs_IMDB)

[('POS_REAL', 'I am a VERY big Jim Carrey fan. I laughed my ASS off during Liar Liar and Ace Ventura. I also like him in his serious movies, especially Truman Show. This one is a cross between his VERY funny side, and his serious side. He is of course VERY funny'), ('POS_REAL', "I haven't seen much German comedy, but if this film is anything to go by, I'm compelled to see more! The simple but effective storyline takes two very different people on a trip from Germany to Italy after Eva, an unemployed mother of two, discovers that her artist husband is"), ('POS_REAL', "There is a scene in Dan in Real Life where the family is competing to see which sex can finish the crossword puzzle first. The answer to one of the clues is Murphy's Law: anything that can go wrong, will go wrong. This is exactly the case for Dan Burns"), ('POS_REAL', 'This was the first regular filmed Columbo movie episode but yet it aired as the second, after Steven Spielberg\'s "Columbo: Murder by the Book". It\'s also 

In [68]:
df = pd.DataFrame(all_seqs_IMDB, columns = ['Source', 'Sequence'])

df.to_csv('/home/ubuntu/real_IMDB.csv', encoding='utf-8', index = False)

In [49]:
!pwd

/home/ubuntu/IMDB_train/neg


In [50]:
!cd ..

In [51]:
!pwd

/home/ubuntu/IMDB_train/neg
